In [53]:
import glob, pandas as pd
data = [{
  'date': pd.to_datetime(filename.split(' ')[0].lstrip('speeches/').lstrip("\\")),
  'speaker': filename.rstrip('.txt').split(' ', 1)[1],
  'text': open(filename, 'rb').readline().decode('cp1252')
} for filename in glob.glob('speeches/*.txt')]
speeches = pd.DataFrame(data)
speeches = speeches.set_index(['speaker', 'date'])
speeches.head()

text
speaker           date                                                         
George Washington 1789-04-30  Fellow Citizens of the Senate and the House of...
                  1789-10-03  Whereas it is the duty of all Nations to ackno...
                  1790-01-08  Fellow Citizens of the Senate and House of Rep...
                  1790-12-08  Fellow citizens of the Senate and House of Rep...
                  1790-12-29  I the President of the United States by my own...

# Task A
Choose a reasonable number of topics for this corpus. One way to think about topics is to consider the number of issues that may have been important in the past as well as those that may have come up over the centuries. Provide a brief explanation of how you chose this number.

In [4]:
N_TOPICS = 15

# Task B
Now perform a topic modeling exercise with LDA. Show the word distributions for each topic as well as topic distributions for each speech. Do you see any shifts over time? Explain.

In [45]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(
  CountVectorizer(stop_words='english'),
  LDA(n_components=N_TOPICS, learning_method='batch', n_jobs=-1, max_iter=25, random_state=42)
)
speech_topic = pipe.fit_transform(speeches.text)

In [46]:
import numpy as np
n_words = 10
vocab = pipe.steps[0][1].get_feature_names()
topic_words = {}
for topic, comp in enumerate(pipe.steps[1][1].components_):   
    word_idx = np.argsort(comp)[::-1][:n_words]
    # store the words most relevant to the topic
    topic_words[topic] = [vocab[i] for i in word_idx]
pd.DataFrame(topic_words).T

,0,1,2,3,4,5,6,7,8,9
0,vietnam,south,president,viet,nam,north,vietnamese,think,people,war
1,president,mr,think,people,going,don,say,time,want,know
2,bank,honduras,banks,justices,rescue,circulation,paper,currency,gold,islands
3,germany,united,conference,german,government,year,great,american,poland,states
4,world,people,peace,nation,great,men,freedom,america,war,new
5,beloved,cherokees,press,gun,boats,newspaper,agent,remonstrance,cattle,information
6,lebanon,israel,country,schedule,army,tariff,middle,men,marines,union
7,thank,stirred,studied,complicated,inconceivable,interpret,familiar,immutable,haste,aspect
8,slavery,law,federal,court,union,case,statute,missouri,question,slave
9,states,government,united,congress,public,country,great,citizens,state,people


In [47]:
import matplotlib.pyplot as plt, seaborn as sns
x = pd.DataFrame(pipe.transform(speeches.text), index=speeches.index)
sns.clustermap(x, cbar=False)

# Task C
In terms of topics addressed “heavily” in a speech, which 3 former presidents does President Trump share the highest similarity with? How did you arrive at your conclusion?

In [54]:
from sklearn.metrics.pairwise import cosine_similarity
xx = x.reset_index(1, True)
mean_topic_per_prez = xx.groupby(xx.index).mean()
similarity = pd.DataFrame(cosine_similarity(mean_topic_per_prez),index = mean_topic_per_prez.index)
similarity.columns = list(mean_topic_per_prez.index)
similarity.loc['Donald Trump'].sort_values(ascending = False)[1:4]

Jimmy Carter             0.674786
Richard Milhous Nixon    0.672095
Lyndon Baines Johnson    0.578314
Name: Donald Trump, dtype: float64

President Trump is most similar to Jimmy Carter, Richard Nixon and LBJ. We arrived at this conclusion by averaging the topic vectors per president and conducting cosine similarities based upon these averages. 

# Task D
In terms of his own speeches, do you see President Trump shifting the emphasis on certain topics over time? Explain your response.

In [55]:
import numpy as np
index = speeches.index
speeches = pd.concat([speeches.reset_index(drop = True),pd.DataFrame(speech_topic)],axis = 1)
speeches.index = index
import matplotlib.pyplot as plt
trump_speeches = speeches.reset_index(inplace = False)
trump_speeches = trump_speeches[trump_speeches.speaker == 'Donald Trump'].sort_values('date')
np.argmax(trump_speeches[list(range(N_TOPICS))].as_matrix(),axis = 1)

array([1, 1, 7, 1, 4, 1], dtype=int64)

In [58]:
trump_speeches[list(range(N_TOPICS))]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
625,0.009524,0.866666,0.009524,0.009524,0.009524,0.009524,0.009524,0.009524,0.009524,0.009524,0.009524,0.009524,0.009524,0.009524,0.009524
627,0.004444,0.861550,0.004444,0.004444,0.004444,0.004444,0.004444,0.004444,0.004444,0.080672,0.004444,0.004444,0.004444,0.004444,0.004444
628,0.007407,0.007407,0.007407,0.007407,0.007407,0.007407,0.007407,0.896296,0.007407,0.007407,0.007407,0.007407,0.007407,0.007407,0.007407
629,0.003922,0.681953,0.003922,0.003922,0.003922,0.003922,0.003922,0.003922,0.003922,0.003922,0.003922,0.003922,0.003922,0.267066,0.003922
630,0.000433,0.200490,0.000433,0.000433,0.699593,0.000433,0.000433,0.000433,0.000433,0.000433,0.000433,0.000433,0.000433,0.094722,0.000433
631,0.005128,0.476663,0.005128,0.005128,0.005128,0.005128,0.005128,0.005128,0.005128,0.005128,0.005128,0.005128,0.005128,0.456670,0.005128


# Task E
If you do a K-means clustering with the same number of clusters as topics, do you see President Trump’s speeches and those of the 3 former presidents you identified in Task C in the same cluster? What was the basis of clustering (e.g., tf-idf, cosine similarity, etc.). Discuss your findings.

# Task F
Provide a visualization of both clusters (with colors) and cosine scores using MDS. 